In [1]:
from name_banker import NameBanker
import pandas as pd
import unittest
from unittest.mock import MagicMock

In [2]:
class TestNameBanker(unittest.TestCase):
    """
    Testcase for namebanker containing uninttests for all its methods
    """
    r = 0.05
    never_returns, always_returns = NameBanker(), NameBanker()
    for i, a in (0, never_returns), (1, always_returns): 
            a.set_interest_rate(r)
            a.predict_proba = MagicMock(return_value=i)
            
    
    def test_set_interest_rate(self):
        banker = NameBanker()
        banker.set_interest_rate(self.r)
        assert banker.rate == self.r
        
    
    def test_predict_proba(self):
        pass
    
    
    def test_fit(self):
        pass
        
        
    def test_expected_utility(self):
        x = pd.Series({"duration": 10, "amount": 100})
        assert self.never_returns.expected_utility(x, 1) < 0, \
            "Utility must be negative if person does not return loan"
        assert self.always_returns.expected_utility(x, 1) > 0, \
            "Utility must be positive if person does not return loan"
    
    
    def test_get_best_action(self):
        for d in range(1, 1000, 10):
            for amt in range(1, 50000, 1500):
                x = pd.Series({"duration": d, "amount": amt})    
                assert not self.never_returns.get_best_action(x), \
                    "When probability of return is 0, best action should always be 0"
                assert self.always_returns.get_best_action(x), \
                    "When probability of return is 1, best action should always be 1"

In [3]:
# We'll need these arguments when running the tests in jupyter notebook
unittest.main(argv=["first-arg-is-ignored"], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 2.714s

OK


Our implementation of NameBanker uses a RandomForestClassifier. It is slow, yet effective.  
A different classifier might be better if training time is critical

In [5]:
# Let's inspect TestLending
!python3 TestLending.py ../../data/credit/D_valid.csv

r=0.017
n_tests=10
100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 10.68it/s]
Testing on class: RandomBanker
	Average utility: -182335.604459863
	Average return on investment: -0.3870209255574014
100%|███████████████████████████████████████████| 10/10 [00:25<00:00,  2.66s/it]
Testing on class: NameBanker
	Average utility: 2225356.6929844273
	Average return on investment: 36.15955010183
